In [0]:
spark.conf.set("fs.azure.account.auth.type.jcy203datalake.dfs.core.windows.net", "ManagedIdentity")
spark.conf.set("fs.azure.account.azure.identity.client.id.jcy203datalake.dfs.core.windows.net", "f75bf38d-183d-4848-963e-b63b1eba1017")

currency_df = spark.read.format("parquet").load("abfss://currency@jcy203datalake.dfs.core.windows.net/")
display(currency_df)